In [1]:
# 数据一
import pandas as pd

def unique_traders(transaction_df, instn_cd):
  # 列举出机构的交易员姓名
  # 创建两个空列表，用于记录买家和卖家名称
  # 创建一个空字典，用于记录交易员的买入和卖出次数
  traders = {}
  # 根据条件提取数据
  # 遍历每一行，计算每个交易员的买入和卖出次数
  for _, row in transaction_df.iterrows():
      if row['byr_instn_cd'] == instn_cd:
          if row['byr_trdr_nm'] in traders:
              traders[row['byr_trdr_nm']]['buy'] += 1
          else:
              traders[row['byr_trdr_nm']] = {'buy': 1, 'sell': 0}
      if row['slr_instn_cd'] == instn_cd:
          if row['slr_trdr_nm'] in traders:
              traders[row['slr_trdr_nm']]['sell'] += 1
          else:
              traders[row['slr_trdr_nm']] = {'buy': 0, 'sell': 1}
  # 获取唯一值
  print("Traders and their buy/sell counts:", traders)
  return traders

def classify_traders(traders_data):
    buy_counts = [trader['buy'] for trader in traders_data.values()]
    sell_counts = [trader['sell'] for trader in traders_data.values()]

    avg_buy = sum(buy_counts) / len(buy_counts)
    avg_sell = sum(sell_counts) / len(sell_counts)

    std_buy = (sum((count - avg_buy) ** 2 for count in buy_counts) / len(buy_counts)) ** 0.5
    std_sell = (sum((count - avg_sell) ** 2 for count in sell_counts) / len(sell_counts)) ** 0.5

    threshold_buy = avg_buy + std_buy*1.5
    threshold_sell = avg_sell + std_sell*1.5

    frequent_traders = {}
    infrequent_traders = {}

    for trader, counts in traders_data.items():
        if counts['buy'] > threshold_buy or counts['sell'] > threshold_sell:
            frequent_traders[trader] = counts
        else:
            infrequent_traders[trader] = counts

    return frequent_traders, infrequent_traders

def filter_transaction_data_by_institution(transaction_df, institution_id):
    """
    根据机构ID筛选交易数据。

    参数:
    - transaction_df (DataFrame): 包含交易数据的DataFrame。
    - institution_id (int): 要筛选的机构ID。

    返回:
    - DataFrame: 筛选后的交易数据。
    """
    transaction_df = transaction_df[(transaction_df['byr_instn_cd'] == institution_id) | (transaction_df['slr_instn_cd'] == institution_id)] 
    return transaction_df

def filter_transaction_data_by_trdr(transaction_df, trdr_nm):
  transaction_df = transaction_df[(transaction_df['byr_trdr_nm'] == trdr_nm) | (transaction_df['slr_trdr_nm'] == trdr_nm)] 
  return transaction_df


In [2]:
# load data
df_transaction = pd.read_csv('../ndm_transaction_list_all_20230705.csv')

# 对于一天以内的所有交易，其中包含多天的产品，需要抽取出其中一只产品的数据
filtered_data = df_transaction[df_transaction['bond_cd'] == 230210]

# selected columns
columns_list = [ "bond_cd", "bnds_nm", "net_prc", "nmnl_vol", "stlmnt_amnt", "byr_instn_cd", "byr_instn_cn_full_nm", "buy_side_trdng_acnt_cd", "byr_trdr_nm", "byr_cptl_acnt_no", "byr_pymnt_sys_cd","slr_instn_cd","slr_instn_cn_full_nm", "slr_trdr_nm"]

# columns_list = [
#     "dl_cd", "dl_tm", "bond_cd", "bnds_nm", "net_prc", "nmnl_vol", "stlmnt_dt", "byr_qt_cd", "byr_instn_cd",
#     "byr_cfets_instn_cd", "byr_instn_cn_full_nm", "byr_trdr_cd", "byr_trdr_nm", "byr_adrs", "byr_trdr_tel",
#     "buy_side_trdng_acnt_cd", "byr_cptl_bnk_nm", "byr_cptl_acnt_no", "byr_pymnt_sys_cd", "byr_dpst_acnt_nm",
#     "buy_side_dpst_cd", "byr_cstdn", "byr_trd_acnt_cfets_cd", "byr_trd_acnt_cn_full_nm", "byr_trd_acnt_cn_shrt_nm",
#     "byr_trd_acnt_en_shrt_nm", "byr_cptl_acnt_nm", "byr_instn_src", "byr_trd_acnt_en_full_nm", "slr_qt_cd",
#     "slr_instn_cd", "slr_cfets_instn_cd", "slr_instn_cn_full_nm", "slr_instn_cn_shrt_nm", "slr_trdr_cd",
#     "slr_trdr_nm", "slr_adrs", "slr_trdr_fax", "slr_lgl_rprsntv", "slr_trdr_tel", "sell_side_trdng_acnt_cd",
#     "slr_trd_sctn", "slr_cptl_bnk_nm", "slr_cptl_acnt_no", "slr_pymnt_sys_cd", "slr_dpst_acnt_nm",
#     "sell_side_dpst_acnt", "slr_cstdn", "slr_trd_acnt_cfets_cd", "slr_trd_acnt_cn_full_nm",
#     "slr_trd_acnt_cn_shrt_nm", "slr_trd_acnt_en_shrt_nm", "slr_cptl_acnt_nm", "slr_trd_acnt_en_full_nm",
#     "slr_agncy_instn_trdr_cd"
# ]

print(columns_list)

# selected df
selected_df = filtered_data[columns_list]
selected_df.to_csv("data_230210.csv", index=False)

['bond_cd', 'bnds_nm', 'net_prc', 'nmnl_vol', 'stlmnt_amnt', 'byr_instn_cd', 'byr_instn_cn_full_nm', 'buy_side_trdng_acnt_cd', 'byr_trdr_nm', 'byr_cptl_acnt_no', 'byr_pymnt_sys_cd', 'slr_instn_cd', 'slr_instn_cn_full_nm', 'slr_trdr_nm']


/var/folders/gj/bwfyx1ks0p17lbgn6fl1nx1c0000gp/T/ipykernel_28699/2197330309.py:2: DtypeWarning: Columns (69,83,110,121,122,123,124,125,128) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transaction = pd.read_csv('../ndm_transaction_list_all_20230705.csv')


In [3]:
def TradingActivity(data):
    # 获取所有买入和卖出的机构ID
    all_ids = set(data['byr_instn_cd'].unique()) | set(data['slr_instn_cd'].unique())

    # 初始化买入和卖出次数的字典
    counts_dict = {'机构ID': [], '机构名称': [], '买入次数': [], '卖出次数': [], '买入量': [], '卖出量': [], '买入花费': [], '卖出花费': []}

    # 遍历每个机构ID
    for institution_id in all_ids:
        # 统计该机构ID的买入次数
        buy_count = len(data[data['byr_instn_cd'] == institution_id])
        buy_volume = data[(data['byr_instn_cd'] == institution_id)]['nmnl_vol'].sum()
        buy_amount = data[(data['byr_instn_cd'] == institution_id)]['stlmnt_amnt'].sum()
        # 统计该机构ID的卖出次数
        sell_count = len(data[data['slr_instn_cd'] == institution_id])
        sell_volume = data[(data['slr_instn_cd'] == institution_id)]['nmnl_vol'].sum()
        sell_amount = data[(data['slr_instn_cd'] == institution_id)]['stlmnt_amnt'].sum()

        # 将统计结果添加到字典中
        counts_dict['机构ID'].append(institution_id)
        counts_dict['买入次数'].append(buy_count)
        counts_dict['卖出次数'].append(sell_count)
        counts_dict['买入量'].append(buy_volume)
        counts_dict['卖出量'].append(sell_volume)
        counts_dict['买入花费'].append(buy_amount)
        counts_dict['卖出花费'].append(sell_amount)
    # 将字典转换为DataFrame
    counts_df = pd.DataFrame(counts_dict)

    # 按照买入次数和卖出次数的大小进行排序
    sorted_df = counts_df.sort_values(by=['买入次数', '卖出次数'], ascending=False)

    # 打印排序后的DataFrame
    print(sorted_df)
    return sorted_df


In [4]:
df_101169 = filter_transaction_data_by_institution(selected_df, 101169)
df_101169_stastics = TradingActivity(df_101169)
df_101169_stastics.to_csv("data_101169_stastics.csv", index=False)

ValueError: All arrays must be of the same length

In [5]:
filter_transaction_data_by_trdr(selected_df, '郭梦晨')

,bond_cd,bnds_nm,net_prc,nmnl_vol,stlmnt_amnt,byr_instn_cd,byr_instn_cn_full_nm,buy_side_trdng_acnt_cd,byr_trdr_nm,byr_cptl_acnt_no,byr_pymnt_sys_cd,slr_instn_cd,slr_instn_cn_full_nm,slr_trdr_nm
2459,230210,23国开10,100.2480,50000000.0,50293508.20,101594,粤开证券股份有限公司,101594,许逸潇,2008020129027304421,102595002014,101169,山西证券股份有限公司,郭梦晨
2989,230210,23国开10,100.2946,10000000.0,10063361.64,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,100359,国信证券股份有限公司,宾馨
3093,230210,23国开10,100.2946,30000000.0,30190084.92,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,301346,江苏江南农村商业银行股份有限公司,朱新军
4451,230210,23国开10,100.2946,30000000.0,30190084.92,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,300889,江苏太仓农村商业银行股份有限公司,谢学荣
4976,230210,23国开10,100.2946,30000000.0,30190084.92,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,301455,东方财富证券股份有限公司,徐鸿峰
5080,230210,23国开10,100.2946,20000000.0,20126723.28,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,102096,万联证券股份有限公司,华海龙
5306,230210,23国开10,100.2946,20000000.0,20126723.28,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,302286,河南济源农村商业银行股份有限公司,黄雅
5361,230210,23国开10,100.2946,20000000.0,20126723.28,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,100374,潍坊银行股份有限公司,何光
5434,230210,23国开10,100.2906,50000000.0,50314808.20,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,101614,鹏华基金管理有限公司,戴东晓
5480,230210,23国开10,100.2946,20000000.0,20126723.28,101169,山西证券股份有限公司,101169,郭梦晨,141000601018170228625,301161000029,301346,江苏江南农村商业银行股份有限公司,章强远


In [6]:
# 示例用法
traders_data = {
    '杨晓晓': {'buy': 78, 'sell': 85},
    '郭梦晨': {'buy': 9, 'sell': 1},
    '荣益': {'buy': 1, 'sell': 1},
    '黄玮嘉': {'buy': 4, 'sell': 3}
}

traders_data = unique_traders(selected_df, 301455)

frequent_traders, infrequent_traders = classify_traders(traders_data)
print("Frequent Traders:", frequent_traders)
print("Infrequent Traders:", infrequent_traders)


Traders and their buy/sell counts: {'郭钰': {'buy': 8, 'sell': 20}, '杨浩天': {'buy': 3, 'sell': 5}, '马宇': {'buy': 7, 'sell': 0}, '徐鸿峰': {'buy': 3, 'sell': 6}, '刘徐琛': {'buy': 1, 'sell': 1}, '张俊朋': {'buy': 0, 'sell': 1}}
Frequent Traders: {'郭钰': {'buy': 8, 'sell': 20}}
Infrequent Traders: {'杨浩天': {'buy': 3, 'sell': 5}, '马宇': {'buy': 7, 'sell': 0}, '徐鸿峰': {'buy': 3, 'sell': 6}, '刘徐琛': {'buy': 1, 'sell': 1}, '张俊朋': {'buy': 0, 'sell': 1}}


In [7]:
def buy_sell_df(transaction_df, institution_id):
    # 初始化操作类型列表
    actions = []
    transaction_df = transaction_df[(transaction_df['byr_instn_cd'] == institution_id) | (transaction_df['slr_instn_cd'] == institution_id)] 
    # 遍历数据，确定每个机构的操作类型
    for index, row in transaction_df.iterrows():
        if row['byr_instn_cd'] == institution_id:
            actions.append('buy')
        if row['slr_instn_cd']  == institution_id:
            actions.append('sell')

    # 将操作类型列表添加到数据中作为新列
    transaction_df['action'] = actions

    # 打印添加操作类型列后的数据
    return transaction_df

In [8]:
df = buy_sell_df(selected_df, 101169)
df = filter_transaction_data_by_trdr(df, '杨晓晓')
features = [ "bond_cd", "bnds_nm", "net_prc", "nmnl_vol","byr_instn_cn_full_nm","byr_trdr_nm","slr_instn_cn_full_nm", "slr_trdr_nm"]
# features = [ "bond_cd", "bnds_nm", "net_prc", "nmnl_vol", "byr_instn_cd", "byr_instn_cn_full_nm", "buy_side_trdng_acnt_cd", "byr_trdr_nm", "byr_cptl_acnt_no", "byr_pymnt_sys_cd","slr_instn_cd","slr_instn_cn_full_nm", "slr_trdr_nm"]
df[features].to_csv('data_shanxi.csv')

/var/folders/gj/bwfyx1ks0p17lbgn6fl1nx1c0000gp/T/ipykernel_28699/3331724395.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df['action'] = actions


In [9]:
df

,bond_cd,bnds_nm,net_prc,nmnl_vol,stlmnt_amnt,byr_instn_cd,byr_instn_cn_full_nm,buy_side_trdng_acnt_cd,byr_trdr_nm,byr_cptl_acnt_no,byr_pymnt_sys_cd,slr_instn_cd,slr_instn_cn_full_nm,slr_trdr_nm,action
88,230210,23国开10,100.3543,10000000.0,1.007010e+07,101169,山西证券股份有限公司,101169,杨晓晓,141000601018170228625,301161000029,101987,江苏昆山农村商业银行股份有限公司,代婧,buy
261,230210,23国开10,100.3583,100000000.0,1.007050e+08,400511,平安基金管理有限公司,411931,都诗扬,000000215002001,901100011115,101169,山西证券股份有限公司,杨晓晓,sell
322,230210,23国开10,100.3583,10000000.0,1.007050e+07,400511,平安基金管理有限公司,460065,赵靖翔,000000629372001,901100011115,101169,山西证券股份有限公司,杨晓晓,sell
370,230210,23国开10,100.2946,50000000.0,5.031681e+07,100011,上海浦东发展银行,100011,宋震,99010135800051002,310290000013,101169,山西证券股份有限公司,杨晓晓,sell
1230,230210,23国开10,100.3762,10000000.0,1.007152e+07,101169,山西证券股份有限公司,101169,杨晓晓,141000601018170228625,301161000029,401208,国寿安保基金管理有限公司,吴梦恬,buy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11806,230210,23国开10,100.2692,100000000.0,1.006082e+08,101169,山西证券股份有限公司,101169,杨晓晓,141000601018170228625,301161000029,101614,鹏华基金管理有限公司,戴东晓,buy
11924,230210,23国开10,100.3756,50000000.0,5.035731e+07,303995,浙江台州椒江农村商业银行股份有限公司,303995,邱婷婷,900030000132001,901100011115,101169,山西证券股份有限公司,杨晓晓,sell
11931,230210,23国开10,100.3802,20000000.0,2.014384e+07,301346,江苏江南农村商业银行股份有限公司,301346,朱新军,01888018224199050008,314304099999,101169,山西证券股份有限公司,杨晓晓,sell
12075,230210,23国开10,100.4144,20000000.0,2.015068e+07,101169,山西证券股份有限公司,101169,杨晓晓,141000601018170228625,301161000029,101987,江苏昆山农村商业银行股份有限公司,刘文涛,buy
